# Lab session 4: Data Warehousing and On-line Analytical Processing

## Introduction 

The aim of this lab (Lab session 4) is for students to get experience with **Data Warehousing** and **On-line Analytical Processing (OLAP)** covered in lecture 5, and more specifically with the concepts of **data cubes**, **data cube measures**, **typical OLAP operations**, and **data cube computation**.

- This lab is the first part of a **two-week assignment** that covers lectures 5 and 6, which is due on **Tuesday 9th November at 10am**.
- The assignment will account for 10% of your overall grade. Questions in this lab sheet (Lab session 4) will contribute to 5% of your overall grade; questions in the sheet of Lab session 5 will cover for another 5% of your overall grade.
- <font color = 'maroon'>The last section of this notebook includes the questions that are assessed towards your final grade.</font> 

## Important notes about the assignment: 

- **PLAGIARISM** <ins>is an irreversible non-negotiable failure in the course</ins> (if in doubt of what constitutes plagiarism, ask!). 
- The total assessed coursework is worth 40% of your final grade.
- There will be 9 lab sessions and 4 assignments.
- One assignment will cover 2 consecutive lab sessions and will be worth 10 marks (percentages of your final grade).
- The submission cut-off date will be 7 days after the deadline and penalties will be applied for late submissions in accordance with the School policy on late submissions.
- You are asked to submit a **report** that should answer the questions specified in the last section of this notebook. The report should be in **PDF format** (so **NOT** *doc, docx, notebook* etc). It should be well identified with your name, student number, assignment number (for instance, Assignment 1), module, and marked with question numbers. 
- No other means of submission other than submitting your assignment through the appropriate QM+ link are acceptable at any time. Submissions sent via email will **not** be considered.
- Please name your report as follows: Assignment2-StudentName-StudentNumber.pdf
- Cases of **Extenuating Circumstances (ECs)** have to go through the proper procedure of the School in due time. Only cases approved by the School in due time can be considered.

## 1. Introduction to Cubes

This chapter describes step-by-step how to use Cubes (http://cubes.databrewery.org/), a lightweight Python framework and set of tools for development of reporting and analytical applications, Online Analytical Processing (OLAP), multidimensional analysis and browsing of aggregated data. We will be working with v1.1 of Cubes. Cubes features:
- a logical view of analysed data - how analysts look at data, how they think of data, not not how the data are physically implemented in the data stores
- OLAP and aggregated browsing (default backend is for relational databse - ROLAP)
- hierarchical dimensions (attributes that have hierarchical dependencies, such as category-subcategory or country-region)
- multiple hierarchies in a dimension
- localizable metadata and data (see Localization)
- authentication and authorization of cubes and their data
- pluggable data warehouse – plug-in other cube-like (multidimensional) data sources

Cubes is meant to be used by application builders that want to provide analytical functionality. Cubes also relies on methods from SQLAlchemy (https://www.sqlalchemy.org/), an open-source SQL toolkit and object-relational mapper for Python.

## 2. Data Preparation

The example data used here is the International Bank for Reconstruction and Development (IBRD) Balance Sheet. For this example we will be using the CSV file "IBRD_Balance_Sheet__FY2010.csv" which is provided in the supplementary material for the lab. The CSV file includes records which are characterised by a Category (and subcategories), Line Item, Fiscal Year, and Amount (in US$ millions). We first start with imports:

In [1]:
# If you are using Google Colab, you would need to run the below line to install Cubes. 
# You can comment the below line if you are running a local python installation with Cubes installed.
!pip install cubes
!pip install sqlalchemy==1.3.20
from sqlalchemy import create_engine
from cubes.tutorial.sql import create_table_from_csv

We can now load the data, create a table and populate it with contents of the CSV file. Note the categories and subcategories created; check the CSV file to link the below commands with the contents of the dataset.

In [2]:
engine = create_engine('sqlite:///data.sqlite')
create_table_from_csv(engine,
                      "IBRD_Balance_Sheet__FY2010.csv",
                      table_name="ibrd_balance",
                      fields=[
                          ("category", "string"),
                          ("category_label", "string"),
                          ("subcategory", "string"),
                          ("subcategory_label", "string"),
                          ("line_item", "string"),
                          ("year", "integer"),
                          ("amount", "integer")],
                      create_id=True
                     )


## 3. Creating a data cube

Everything in Cubes happens in an *analytical workspace*. It contains cubes, maintains connections to the data stores (with cube data), provides connection to external cubes and more.

The workspace properties are specified in a configuration file slicer.ini (default name). The first thing we have to do is to specify a data store – a database which will host the cube’s data:

In [3]:
from cubes import Workspace

workspace = Workspace()
workspace.register_default_store("sql", url="sqlite:///data.sqlite")

The structure of data cubes (in terms of dimensions, measures, and aggregates) is specified in JSON files. We now import file 'tutorial_model.json' (found in the lab supplementary material) which includes an example model of the data cube, dimension tables, and aggregate functions for the CSV file we loaded previously.

In [4]:
workspace.import_model("tutorial_model.json")

**Please make sure to inspect the structure of the above JSON file - this will be relevant for one of the assignment questions.**

We can now create a data cube based on the above data cube model and data table:

In [5]:
cube = workspace.cube("ibrd_balance")
print("success")

success


## 4. Aggregations and OLAP operations

*Browser* is an object that does the actual aggregations and other data queries for a cube. To obtain one:

In [6]:
browser = workspace.browser(cube)

We can now compute aggregates of the data cube as specified by the data cube model. For computing the total count of records:

In [7]:
result = browser.aggregate()
result.summary["record_count"]

62

For computing a sum of the amount:

In [15]:
result.summary["amount_sum"]

550840

Now we can try to compute aggregates by year:

In [70]:
result = browser.aggregate(drilldown=["year"])
for record in result:
    print(record)

{'year': 2009, 'min_amount': -1683, 'max_amount': 110040, 'amount_sum': 550840, 'record_count': 31}
{'year': 2010, 'min_amount': -3043, 'max_amount': 128577, 'amount_sum': 566020, 'record_count': 31}


Or compute aggregates by item category:

In [173]:
result = browser.aggregate(drilldown=["item:subcategory"])
print(result.summary)
for record in result:
    print(record)

{'min_amount': -3043, 'max_amount': 128577, 'amount_sum': 1116860, 'record_count': 62}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'da', 'item.subcategory_label': 'Derivative Assets', 'min_amount': 2246, 'max_amount': 87457, 'amount_sum': 244691, 'record_count': 8}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'dfb', 'item.subcategory_label': 'Due from Banks', 'min_amount': 222, 'max_amount': 2380, 'amount_sum': 4847, 'record_count': 4}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'i', 'item.subcategory_label': 'Investments', 'min_amount': 36012, 'max_amount': 41012, 'amount_sum': 77024, 'record_count': 2}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'lo', 'item.subcategory_label': 'Loans Outstanding', 'min_amount': 103657, 'max_amount': 118104, 'amount_sum': 221761, 'record_count': 2}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'nn

We can also perform *slicing* and *dicing* operations on the data cube. The below example performs a slicing operation on the data cube by selecting only entries with the year being 2009, and displays aggregates according to the item category. Here, a *cell* defines a point of interest – portion of the cube to be aggergated or browsed.

In [99]:
import cubes as cubes
cuts = [cubes.PointCut("year", ["2009"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell, drilldown=["item"])
for record in result:
    print(record)

{'item.category': 'a', 'item.category_label': 'Assets', 'min_amount': 33, 'max_amount': 103657, 'amount_sum': 275420, 'record_count': 16}
{'item.category': 'e', 'item.category_label': 'Equity', 'min_amount': -1683, 'max_amount': 29870, 'amount_sum': 40037, 'record_count': 4}
{'item.category': 'l', 'item.category_label': 'Liabilities', 'min_amount': 57, 'max_amount': 110040, 'amount_sum': 235383, 'record_count': 11}


It's worth noting that in Cubes, slicing operations can be created by either specifying a "point cut" which selects a single value of an attribute in a given dimension (called using the cubes.PointCut() function as above), or by specifying a "range cut", which selects a range of values for a given dimension. The range cut can be called using the cubes.RangeCut() function, which takes as input the attribute name, the minimum value of the specified range, and the maximum value of the range.

Similarly, we can perform a *dicing* operation on the data cube by performing a selection on two or more dimensions. The below example performs a dicing operation on the data cube, selecting entries with the year being 2009 and the item category being "a", and displays the aggregate results:

In [100]:
cuts = [cubes.PointCut("year", ["2009"]),cubes.PointCut("item", ["a"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell,drilldown=["item"])
result.summary

{'min_amount': 33,
 'max_amount': 103657,
 'amount_sum': 275420,
 'record_count': 16}

We can also *drill down* lower in the Category hierarchy. Here, we perform a dicing operation to select records with year being 2009 and item category being "a" (corresponding to assets), and show aggregates for each subcategory level.

In [101]:
cuts = [cubes.PointCut("year", ["2009"]),cubes.PointCut("item", ["a"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell,drilldown=["item:subcategory"])
for record in result:
    print(record)

{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'da', 'item.subcategory_label': 'Derivative Assets', 'min_amount': 2246, 'max_amount': 82793, 'amount_sum': 123065, 'record_count': 4}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'dfb', 'item.subcategory_label': 'Due from Banks', 'min_amount': 664, 'max_amount': 2380, 'amount_sum': 3044, 'record_count': 2}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'i', 'item.subcategory_label': 'Investments', 'min_amount': 41012, 'max_amount': 41012, 'amount_sum': 41012, 'record_count': 1}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'lo', 'item.subcategory_label': 'Loans Outstanding', 'min_amount': 103657, 'max_amount': 103657, 'amount_sum': 103657, 'record_count': 1}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'nn', 'item.subcategory_label': 'Nonnegotiable', 'min_amount': 1202, 'max_amount': 1202, '

## <font color = 'maroon'>Assignment #2 (Part 1)</font>

Question 1 is a pen-and-paper exercise; questions 2 and 3 are coding exercises. For all your answers to the assignment, please include your workings (e.g., equations, code) when this is relevant to the question.

1. Suppose that a car dealer company has a data warehouse that holds record ID lists of vehicles in terms of brands (Ferrari, Maserati, Lamborghini) and store branches (Rome, Milan, Naples). Each record consists of a combination of vehicle brand and branch. We would like to index the OLAP data using bitmap indices. Produce the *base table* for record IDs, and the corresponding *bitmap index tables* for vehicle brand and store branch. <font color = 'maroon'>[1 mark out of 5]</font>
  

2. Using the same CSV file and data cube in the above lab tutorial, modify the "tutorial_model.json" file to include aggregate measures for the minimum and maximum amount in the data cube. Using these implemented aggregate measures, produce the values for the minimum and maximum amount in the data per year. Make sure to show your workings in the PDF report. <font color = 'maroon'>[2 marks out of 5]</font>


3. Using the CSV file "country-income.csv" (found in the supplementary lab documents), perform the following:
  1. Load the CSV file using Cubes, create a JSON file for the data cube model, and create a data cube for the data. Use as dimensions the region, age, and online shopper fields. Use as measure the income. Define aggregate functions in the data cube model for the total, average, minimum, and maximum income. In your PDF report, show the relevant scripts and files created. <font color = 'maroon'>[1 mark out of 5]</font>
  2. Using the created data cube and data cube model, produce aggregate results for: the whole data cube; results per region; results per online shopping activity; and results for all people aged between 40 and 50. <font color = 'maroon'>[1 mark out of 5]</font>
  

In [23]:
#Q2

import cubes as cubes
cuts2009 = [cubes.PointCut("year", ["2009"])]
cell2009 = cubes.Cell(cube, cuts2009)
print(cell2009)
min2009 = browser.aggregate(cell2009).summary["min_amount"]
max2009 = browser.aggregate(cell2009).summary["max_amount"]
print(min2009)
print(max2009)

cuts2010 = [cubes.PointCut("year", ["2010"])]
cell2010 = cubes.Cell(cube, cuts2010)
print(cell2010)
min2010 = browser.aggregate(cell2010).summary["min_amount"]
max2010 = browser.aggregate(cell2010).summary["max_amount"]
print(min2010)
print(max2010)


year:2009
-1683
110040
year:2010
-3043
128577


In [119]:
#Q4

engine2 = create_engine('sqlite:///data.sqlite')
create_table_from_csv(engine2,
                      "country-income.csv",
                      table_name="country-income",
                      fields=[
                          ("region", "string"),
                          ("age", "integer"),
                          ("income", "integer"),
                          ("online_shopper", "string")],
                      create_id=True
                     )

workspace2 = Workspace()
workspace2.register_default_store("sql", url="sqlite:///data.sqlite")
workspace2.import_model("country-model.json")

cube2 = workspace2.cube("country-income")

browser2 = workspace2.browser(cube2)

In [172]:
#Q5

#whole data set
full_data = browser2.aggregate()
print("Full data summary")
print(full_data.summary)

#results per region
region_data = browser2.aggregate(drilldown = ["region"])
print("\nResults per region")
for i in region_data:
    print(i)
    
#results per shopping activity
print("\nResults per shopping activity")

result = browser.aggregate(drilldown=["item:subcategory"])
print(result.summary)
for record in result:
    print(record)

cutsShop = [cubes.RangeCut("person", ["0"],["100"])]
cellShop = cubes.Cell(cube2, cutsShop)
shopv2 = browser2.aggregate(cellShop, drilldown=["person:online_shopper"])
print(shopv2.summary)

shopping_data = browser2.aggregate(drilldown=["person:online_shopper"])
for j in shopping_data:
    print(j)
    

#results for people aged 40-50
print("\nResults per age")
cutsAge = [cubes.RangeCut("person", ["40"],["50"])]
cellAge = cubes.Cell(cube2, cutsAge)
age_data = browser2.aggregate(cellAge, drilldown=["person"])
for l in age_data:
    print(l)

Full data summary
{'min_income': 57600, 'max_income': 99600, 'income_sum': 768200, 'record_count': 10, 'avg_income': 76820.0}

Results per region
{'region': 'Brazil', 'min_income': 57600, 'max_income': 73200, 'income_sum': 193200, 'record_count': 3, 'avg_income': 64400.0}
{'region': 'India', 'min_income': 69600, 'max_income': 94800, 'income_sum': 331200, 'record_count': 4, 'avg_income': 82800.0}
{'region': 'USA', 'min_income': 64800, 'max_income': 99600, 'income_sum': 243800, 'record_count': 3, 'avg_income': 81266.66666666667}

Results per shopping activity
{'min_income': 57600, 'max_income': 99600, 'income_sum': 768200, 'record_count': 10, 'avg_income': 76820.0}
{'person.age': 32, 'person.online_shopper': 'Yes', 'min_income': 57600, 'max_income': 57600, 'income_sum': 57600, 'record_count': 1, 'avg_income': 57600.0}
{'person.age': 35, 'person.online_shopper': 'No', 'min_income': 64800, 'max_income': 64800, 'income_sum': 64800, 'record_count': 1, 'avg_income': 64800.0}
{'person.age': 40